In [ ]:
import datasets
from src import data_loading
from src.utils.timer import Timer

Timer.default_verbosity = "none"

dataset = datasets.load_dataset("lmarena-ai/arena-human-preference-140k")
data = data_loading.load_training_data(dataset["train"].to_pandas())
print(f"Successfully loaded {len(data.entries)} entries")

In [ ]:
from src.models.dense_network_model import DenseNetworkModel
from src.models.optimizers.adamw_spec import AdamWSpec
from src.utils.data_split import ValidationSplit

model = DenseNetworkModel(
    embedding_model_name="all-mpnet-base-v2",
    hidden_dims=[512, 512, 256, 256, 128, 128, 64, 64],
    model_id_embedding_dim=128,
    optimizer_spec=AdamWSpec(
        learning_rate=5e-5,
        lr_decay_gamma=0.998,
    ),
    balance_model_samples=True,
    print_every=5,
)

model.train(data, validation_split=ValidationSplit(val_fraction=0.2, seed=42), epochs=120, batch_size=512)
model.save("short-run-with-all-mpnet-base-v2")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from src.plotting_utils import plot_loss, plot_accuracy

history = model.get_history()
_, axes = plt.subplots(2, 2, figsize=(15, 10))

plot_loss(axes[0, 0], history.total_loss, "Training loss")
plot_loss(axes[0, 1], history.val_loss, "Validation loss")
plot_accuracy(axes[1, 0], history.train_accuracy, "Training accuracy")
plot_accuracy(axes[1, 1], history.val_accuracy, "Validation accuracy")

plt.tight_layout()
plt.show()

In [ ]:
from src.data_models.data_models import InputData
from src.preprocessing.prompt_embedding_preprocessor import PromptEmbeddingPreprocessor
from src.utils import data_split

_, validation_prompts = data_split.train_val_split(PromptEmbeddingPreprocessor.filter_out(data), val_fraction=0.2, seed=42)
results = model.predict(InputData(
        prompts=validation_prompts,
        model_names=model._model_encoder.names,
    ),
    batch_size=512,
).scores

sorted_results = sorted(
    [(name, float(np.mean(scores)), float(np.std(scores))) for name, scores in results.items()],
    key=lambda x: x[1],
    reverse=True
)

for name, mean_score, std_score in sorted_results:
    print(f"{(name+":"):<{max(len(name) for name, _, _ in sorted_results)+1}s} {mean_score:.6f} ± {std_score:.6f}")